In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import warnings
import sys
import time
warnings.filterwarnings('ignore')

from tqdm import tqdm
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

In [ ]:
try:
    import google.colab
    from google.colab import drive
    !uv pip install anomalib
    !uv pip install open-clip-torch
    drive.mount('/content/drive', force_remount=True)
    PROJECT_ROOT = Path('/content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation') # 본인 경로 수정
except ImportError:
    PROJECT_ROOT = Path.cwd().parents[1]

os.chdir(PROJECT_ROOT)
print(f"Current working directory: {os.getcwd()}")

Using Python 3.12.12 environment at: /usr
Resolved 94 packages in 1.56s
Prepared 12 packages in 498ms
Installed 12 packages in 17ms
 + anomalib==2.2.0
 + freia==0.2
 + imagecodecs==2026.1.14
 + jsonargparse==4.46.0
 + kornia==0.8.2
 + kornia-rs==0.1.10
 + lightning==2.6.1
 + lightning-utilities==0.15.2
 + pytorch-lightning==2.6.1
 + rich-argparse==1.7.2
 + torchmetrics==1.8.2
 + typeshed-client==2.8.2
Using Python 3.12.12 environment at: /usr
Resolved 49 packages in 115ms
Prepared 2 packages in 44ms
Installed 2 packages in 4ms
 + ftfy==6.3.1
 + open-clip-torch==3.2.0
Mounted at /content/drive
Current working directory: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation


### 경로 설정

In [ ]:
# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"

# output path
OUTPUT_ROOT = PROJECT_ROOT / "output"

print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Root: {DATA_ROOT}")
print(f"Config Root: {RUNTIME_CONFIG_ROOT}")

Project Root: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation
Data Root: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD
Config Root: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/configs/runtime.yaml


### Config 로드 & 오버라이드

In [ ]:
from src.utils import load_config

# tqdm bar off
os.environ["TQDM_DISABLE"] = "1"

config = load_config(RUNTIME_CONFIG_ROOT)

# EfficientAD로 모델 오버라이드
config["anomaly"]["model"] = "efficientad"
print(f"Model: {config['anomaly']['model']}")
config

Model: efficientad


{'data': {'root': 'dataset/MMAD', 'output_root': './output'},
 'anomaly': {'model': 'efficientad',
  'patchcore': {'backbone': None,
   'layers': None,
   'coreset_sampling_ratio': None},
  'winclip': {'class_name': None, 'k_shot': None, 'scales': None},
  'efficientad': {'model_size': None, 'teacher_out_channels': None}},
 'training': {'train_batch_size': None,
  'eval_batch_size': None,
  'num_workers': None,
  'max_epochs': None},
 'engine': {'accelerator': 'gpu',
  'enable_progress_bar': False,
  'logger': False},
 'output': {'save_json': True}}

### Train

In [7]:
from scripts.train_anomalib import Anomalibs
from src.datasets.dataloader import MMADLoader

# MVTec-LOCO + GoodsAD만 학습
TARGET_DATASETS = ["MVTec-LOCO", "GoodsAD"]

runner = Anomalibs()
# 모델을 efficientad로 오버라이드
runner.model_name = "efficientad"
# EfficientAD는 train_batch_size=1 필수
runner.training_config["train_batch_size"] = 1
# 데이터셋 한정
runner.loader.DATASETS = TARGET_DATASETS

# 대상 카테고리 확인
categories = runner.get_all_categories()
print(f"Total categories: {len(categories)}")
for ds, cat in categories:
    print(f"  {ds}/{cat}")

TrainAnomalib - INFO - Initialized - model: efficientad, device: cuda
INFO:TrainAnomalib:Initialized - model: efficientad, device: cuda


Device: NVIDIA A100-SXM4-80GB
Total categories: 11
  MVTec-LOCO/breakfast_box
  MVTec-LOCO/juice_bottle
  MVTec-LOCO/pushpins
  MVTec-LOCO/screw_bag
  MVTec-LOCO/splicing_connectors
  GoodsAD/cigarette_box
  GoodsAD/drink_bottle
  GoodsAD/drink_can
  GoodsAD/food_bottle
  GoodsAD/food_box
  GoodsAD/food_package


In [8]:
runner.fit_all()

TrainAnomalib - INFO - Starting fit_all: 11 categories
INFO:TrainAnomalib:Starting fit_all: 11 categories
TrainAnomalib - INFO - [1/11] MVTec-LOCO/breakfast_box
INFO:TrainAnomalib:[1/11] MVTec-LOCO/breakfast_box
TrainAnomalib - INFO - Fitting efficientad - MVTec-LOCO/breakfast_box
INFO:TrainAnomalib:Fitting efficientad - MVTec-LOCO/breakfast_box
INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor  │ PreProcessor     │      0 │ train │     0 │
│ 1 │ post_processor │ PostProcessor    │      0 │ train │     0 │
│ 2 │ evaluator      │ Evaluator        │      0 │ train │     0 │
│ 3 │ model          │ EfficientAdModel │  8.1 M │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 8.1 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 8.1 M                                                                                                
Total estimated model params size (MB): 32                                                                         
Modules in train mode: 48                                                                                          
Modules in eval mode: 7                                                                                            
Total FLOPs: 0

efficientad_pretrained_weights.zip: 40.0MB [00:02, 19.6MB/s]                            
imagenette2.tgz: 1.56GB [01:49, 14.2MB/s]                            
Calculate Validation Dataset Quantiles: 100%|██████████| 6/6 [00:06<00:00,  1.05s/it]
INFO:lightning_fabric.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/trainer.py", line 630, in _fit_impl
    self._run(model, ckpt_path=ckpt_path, weights_only=weights_only)
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/trainer.py", line 1079, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/trainer.py", line 1123, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/fit_loop.py", line 217, in run
    self.advance()
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/fit_loop.py", line 465, in advance
    self.epoch_loop.run(self._data_fetcher)
  File

TypeError: object of type 'NoneType' has no len()

### Predict

In [ ]:
all_predictions = runner.predict_all()

### Metrics 계산

In [ ]:
import torch
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    jaccard_score  # IoU
)

results = []
for category, preds in all_predictions.items():
    # Image-level
    y_true = np.concatenate([p.gt_label.cpu().numpy() for p in preds])
    y_score = np.concatenate([p.pred_score.cpu().numpy() for p in preds])
    y_pred = (y_score >= 0.5).astype(int)

    metrics = {
        "Category": category,
        "AUROC": round(roc_auc_score(y_true, y_score), 4),
        "Accuracy": round(accuracy_score(y_true, y_pred), 4),
        "Precision": round(precision_score(y_true, y_pred, zero_division=0), 4),
        "Recall": round(recall_score(y_true, y_pred, zero_division=0), 4),
        "F1": round(f1_score(y_true, y_pred, zero_division=0), 4),
        "N_samples": len(y_true)
    }

    # Pixel-level: gt_mask가 유효한 경우에만 계산
    # (Folder로 로드 시 mask_dir 제외하면 gt_mask가 전부 0)
    gt_masks = torch.cat([p.gt_mask for p in preds]).int()
    has_gt_mask = gt_masks.sum() > 0
    if has_gt_mask:
        pred_masks = torch.cat([(p.anomaly_map > 0.5).int() for p in preds])
        metrics["Dice"] = round(
            f1_score(
                gt_masks.flatten().cpu().numpy(),
                pred_masks.flatten().cpu().numpy(),
                zero_division=0
            ),
            4
        )
        metrics["IoU"] = round(
            jaccard_score(
                gt_masks.flatten().numpy(),
                pred_masks.flatten().numpy(),
                average='binary',
                zero_division=0
            ),
            4
        )
    else:
        metrics["Dice"] = None
        metrics["IoU"] = None

    results.append(metrics)

metrics_df = pd.DataFrame(results).set_index("Category")
numeric_cols = metrics_df.select_dtypes(include='number').columns
avg_row = metrics_df[numeric_cols].drop(columns=['N_samples']).mean().round(4)
avg_row['N_samples'] = metrics_df['N_samples'].sum()
metrics_df.loc['Average'] = avg_row

metrics_df

### 시각화

In [ ]:
from src.visual.plot import heatmap_plot

metrics_trans = metrics_df.drop('Average').drop(columns='N_samples')
# Dice/IoU가 전부 None이면 해당 컬럼 제거
metrics_trans = metrics_trans.dropna(axis=1, how='all')

heatmap_plot(
    metrics_trans,
    figsize=(10, 10),
    cmap='RdYlGn',
    annot=True,
    fmt='.4f',
    linewidths=0.5,
    title='EfficientAD Performance (MVTec-LOCO + GoodsAD)',
    rotation_x=45,
    rotation_y=0
)

In [ ]:
from src.visual.plot import anomaly_grid_from_dir

for ds, cat in categories:
    OUTPUT_IMG_ROOT = OUTPUT_ROOT / "Efficientad" / ds / cat / "latest" / "images"
    print(f"{ds}/{cat}")
    anomaly_grid_from_dir(OUTPUT_IMG_ROOT, n_samples=1, n_cols=1)